# ESML - accelerator: Batch scoring pipeline
- 1) `AutoMap datalake` & init ESML project
- 2) `Get earlier trained model`
- 3) `Score with GOLD_TEST` and calculate ML-performance




######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

In [1]:
unregister_all_datasets=False

In [2]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.inference_mode = True

if(unregister_all_datasets):
    p.unregister_all_datasets(p.ws) # For DEMO purpose


In [3]:
print("Inference mode (otherwise Training mode):", p.inference_mode)
p.describe()

Inference mode (otherwise Training mode): True
Inference version: 4

 - ds01_titanic
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/in/dev/2021/06/16/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/bronze/dev/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/silver/dev/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/in/dev/2021/06/16/
 

Training GOLD (p.GoldPath)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_17/5da0f00f3c6d4a34989de39f7b44ffdb/
 

A)INFERENCE ONLINE: GOLD scored (example if realtime today)
projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_17/5da0f00f3c6d4a34989de39f7b44ffdb/
 

[B) USAGE]: to_score_folder_batch, scored_

In [14]:
datastore = p.init() # 3) Automapping from datalake to Azure ML datasets

...
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = FALSE. [active_in_folder.json,active_scoring_in_folder.json] not found. 
 - Using [active_in_folder.json,active_scoring_in_folder.json] from ArgParse or GIT. No override from datalake settings

Inference mode (False = Training mode): True
Load data as Datasets....
ds01_titanic

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 1 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002 

Dataset 'ds01_titanic' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######


#  SCORING PIPELINE - `ESML AutoLake paths` for SCORING

# 1) REAL-TIME & ONLINE AKS webservice
- No data exists in IN-folder as for BATCH scoring, data comes as GOLD from REST call, but is saved automatically by ESML in "GOLD to_score" (for lineage purpose)
    - Assuming, Bronze2Gold is taken care of by another pipeline (ADF or AML)
- WriteBack function: Use the 2nd print out, If you want to create a "daily report" or writeback data to a "source" with the prediction. Read this scored data from ESML AutoLake

In [5]:
to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
print("A)TO SCORE -  INFERENCE ONLINE: GOLD to score (example if realtime - today)")
print(to_score_folder)
print("")
print(" ...above reading, is not required for ONLINE (data via REST), but good for lineage - to save REST-data, see what data was scored by the AKS-endpoint")
print("")
print("A)SCORED - INFERENCE ONLINE: GOLD scored (example if realtime today)")
print("")
print(scored_folder)
print("")

A)TO SCORE -  INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_17/a896d979d55b42edbd9f9c2cd8f2d54d/

 ...above reading, is not required for ONLINE (data via REST), but good for lineage - to save REST-data, see what data was scored by the AKS-endpoint

A)SCORED - INFERENCE ONLINE: GOLD scored (example if realtime today)

projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_17/a896d979d55b42edbd9f9c2cd8f2d54d/



# 2) BATCH SCORING `IN -> Bronze->Silver`
- BATCH? REALTIME / ONLINE scoring with AKS, can also score multiple rows, but if there's a lot of data to score or ONLINE webservice is not needed, use this approach instead.
- Bronze2Gold: Refinement is needed from IN->Bronze-Silver for each dataset, then merge all to 1 GOLD. 
    - This procedure can be a `pipeline` (ADF or AML) or a `piplinestep`. Example of the latter: The 1st AML `Bronze2Gold`-piplinestep before a 2nd `ScoringStep`
    - Last action is to MERGE all silver-states to 1 GOLD dataset, which another pipeline (or pipelinestep) are targeting

## 2A) ESML auotomatically saves "to_score" and "scored" - `aligend to Datalake design`
- ESML Feture engineering: `Bronze 2 Gold` for inference
- `Benefits`: User `do not need to remember any paths` in datalake
- -Cons: created an ESML SDK dependancy in the Azure Databricks notebook, or Azure ML Bronze2Gold Pipeline

In [15]:
print(p.GoldPathToScoreBatch) # This is the GOAL. After Bronze2Gold we have this folder

projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/


## `Bronze2Gold` - prep data `to_score`

In [19]:
# Feture engineering: Bronze 2 Gold - working with Azure ML Datasets with Bronze, Silver, Gold concept
esml_dataset = p.DatasetByName("ds01_titanic") # Get dataset
df_bronze = esml_dataset.Bronze.to_pandas_dataframe()
p.save_silver(esml_dataset,df_bronze) #Bronze -> Silver

df = esml_dataset.Silver.to_pandas_dataframe() 
gold_to_score, to_score_version_folder = p.save_gold(df)  #Silver -> Gold

print(p.GoldPathToScoreBatch)
print(to_score_version_folder)

projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/94b3af25c2014f90bfed2af07d155664/


In [20]:
print(p.GoldPathToScoreBatch + '*/*.parquet')
print(to_score_version_folder)

projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/*/*.parquet
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/94b3af25c2014f90bfed2af07d155664/


In [21]:
from azureml.core.dataset import Dataset
Dataset.get_by_name(workspace = p.ws, name =  "M01_GOLD_TO_SCORE")

{
  "source": [
    "('project002', 'projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/94b3af25c2014f90bfed2af07d155664/*.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "4723785c-d4ff-4741-a7ac-4612269bd1f7",
    "name": "M01_GOLD_TO_SCORE",
    "version": 2,
    "description": "01_titanic_model_clas: GOLD_to_score.parquet",
    "tags": {
      "date_time_folder": "2021_06_16",
      "model_version": "4",
      "model": "01_titanic_model_clas"
    },
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}

In [22]:
print(p.Gold)

TabularDataset
{
  "source": [
    "('project002', 'projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/94b3af25c2014f90bfed2af07d155664/*.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "4723785c-d4ff-4741-a7ac-4612269bd1f7",
    "name": "M01_GOLD_TO_SCORE",
    "version": 2,
    "description": "01_titanic_model_clas: GOLD_to_score.parquet",
    "tags": {
      "date_time_folder": "2021_06_16",
      "model_version": "4",
      "model": "01_titanic_model_clas"
    },
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}


In [23]:
from azureml.core.dataset import Dataset
ds = Dataset.get_by_name(workspace = p.ws, name =  p.Gold.name) # Opton 1) Azure ML Dataset directly
ds = Dataset.get_by_name(workspace = p.ws, name =  "M01_GOLD_TO_SCORE") # Opton 2) This would be static for this model, always same, fetches latest version


if(ds.name != p.Gold.name): # Opton 2) ESMLProject.Gold property
    print ("error")
else: 
    print ("2 ways to find same Dataset: {}".format( p.Gold.name))
    print("Q: OK. Yes, the path exists, including GUID-folder in Azure ML: ", to_score_version_folder in str(p.Gold))

2 ways to find same Dataset: M01_GOLD_TO_SCORE
Q: OK. Yes, the path exists, including GUID-folder in Azure ML:  True


## 2B) Manually "to_score" and "scored" - still `alignd to Datalake design` (with help from ESML)
- Azure ML Feture engineering: `Bronze 2 Gold` for inference, but with Azure ML Datasets directly (not via ESML)
- `Benefits`: No ESML SDK dependancy to DEFINE the pipepline "Pythoncript" (to execute it, ESML never is needed).
- Cons: 
    - You cannot user ESML Bronze2Gold concept in your Azure ML Bronze2Gold Pipeline (since not installed), you need to:
    - Manually write dynamic data-paths to Datalake GEN 2 (work with `date_folder paths`, `model_version` `dev,test,prod`, `datasets`, `bronze,silver, gold` (which ESML still can help genereate) 
    - Manually register your Datasets manually in Azure ML Studio, with versioning, and with naming convention on Dataset names, and manually TAG datasets with metadata (date_folder, etc)


In [17]:
p.inference_mode = True # This flag will "change the paths", from TRAIN folder to INFERENCE folder-structure

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath)
    print("Bronze", d.BronzePath)
    print("Silver", d.SilverPath)

# MANUALLY - Get help Generate 2 paths...use them to manually save "to_score" and "scored"
to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder) 
print()
print("Gold to SCORE: ", to_score_folder, "  ...uuid folder, is to be able to have multiple unique scorings, same datetime")
print()
print("...bronze2Gold here, manually upload to datalake  (manually register dataset if needed)")
print()
print("Gold to SCORED: ", scored_folder)


ds01_titanic
IN projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/in/dev/2021/06/16/
Bronze projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/bronze/dev/
Silver projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/silver/dev/

Gold to SCORE:  projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/e698211ce52f4d0bbfef67f6ed40b83b/   ...uuid folder, is to be able to have multiple unique scorings, same datetime

...bronze2Gold here, manually upload to datalake  (manually register dataset if needed)

Gold to SCORED:  projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_16/e698211ce52f4d0bbfef67f6ed40b83b/


# `TRAIN` example (p.`inference_mode = False`)

In [33]:
p.inference_mode = False # Example for flag set to TRAIN mode

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath) # datefolder, reads from Datalake Active folder
    print("Bronze", d.BronzePath) # intermediate
    print("Silver", d.SilverPath) # intermediate

print("Gold TRAIN", p.GoldPath + 'Train/*/', "*.parquet") # Gold/Env/Train/versionfolder/data.parquet
print("Gold TRAIN", p.GoldTrain) 

ds01_titanic
IN projects/project002/01_titanic_model_clas/train/ds01_titanic/in/dev/2021/01/16/
Bronze projects/project002/01_titanic_model_clas/train/ds01_titanic/out/bronze/dev/
Silver projects/project002/01_titanic_model_clas/train/ds01_titanic/out/silver/dev/
Gold TRAIN projects/project002/01_titanic_model_clas/train/gold/dev/Train/*/ *.parquet
Gold TRAIN TabularDataset
{
  "source": [
    "('project002', 'projects/project002/01_titanic_model_clas/train/gold/dev/Train/5b37804fff294e1b93b2bd9fa763e2fa/*.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "d646ef17-1159-4a5a-9b0f-7b20aebe2a9d",
    "name": "M01_GOLD_TRAIN",
    "version": 1,
    "description": "01_titanic_model_clasGOLD_TRAIN.parquet from splitted Train, Validate, Test",
    "tags": {
      "split_percentage": "0.6",
      "label": "Survived",
      "model": "01_titanic_model_clas"
    },
    "workspace": "Workspace.create(name='msft-w

In [34]:
from azureml.core.dataset import Dataset 
ds1 = Dataset.get_by_name(workspace = p.ws, name =  p.Gold.name) # Opton 1) Azure ML Dataset directly
ds2 = Dataset.get_by_name(workspace = p.ws, name = "M01_GOLD_TO_SCORE") # Opton 2) This would be static for this model, fetches latest version

if(ds1.name != p.Gold.name and ds2.name != p.Gold.name): # Opton 3) ESMLProject.Gold property
    print ("error", ds2.name)
    print ("error", p.Gold.name)
else: 
    print ("2 ways to find same Dataset: {}".format( p.Gold.name))
    print("Q: OK. Yes, the path exists, including GUID-folder in Azure ML: ", to_score_version_folder in str(p.Gold))

2 ways to find same Dataset: M01_GOLD
Q: OK. Yes, the path exists, including GUID-folder in Azure ML:  False


# BATCH SCORING `GOLD` ("to_score" and "scored" folders)
- Assuming another pipeline (ADF or AML) has already refined data from `IN->Bronze->Silver->Gold`

In [35]:
print("A)INFERENCE BATCH - ALL scorings that day (datetime from config)")
print(p.GoldPathToScoreBatch + '*/*.parquet')
print("")
print("B)INFERENCE BATCH - LATEST scoring that day")
print(p.GoldPathToScoreBatch + 'gold_to_score.parquet')
print("")
print("")

A)INFERENCE BATCH - ALL scorings that day (datetime from config)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/*/*.parquet

B)INFERENCE BATCH - LATEST scoring that day
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/gold_to_score.parquet




In [36]:
to_score_folder_batch, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder)

print("C)INFERENCE BATCH: SPECIFIC scoring, that day (generate paths to simulate, datetime from config)")
print(to_score_folder_batch + 'gold_to_score.parquet') # + self.date_scoring_folder.strftime('%Y_%m_%d') + '/')
print("")
print("D)INFERENCE BATCH: GOLD scored (example batch, datetime from config)")
print(scored_folder +  'scored_*.parquet') 
print("")
print("...The UUID folder at the end, MAIN PURPOSE, to enable multiple unique scorings same day. But can be overridden with a surrogate key, e.g. customerID")

C)INFERENCE BATCH: SPECIFIC scoring, that day (generate paths to simulate, datetime from config)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/5096b4e54d344aa290f8402cb78f7d33/gold_to_score.parquet

D)INFERENCE BATCH: GOLD scored (example batch, datetime from config)
projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_16/5096b4e54d344aa290f8402cb78f7d33/scored_*.parquet

...The UUID folder at the end, MAIN PURPOSE, to enable multiple unique scorings same day. But can be overridden with a surrogate key, e.g. customerID


In [34]:
import uuid
my_unique_folder_customerID = uuid.uuid4().hex

In [56]:
# Using a STATIC uuid, can be a "sytem_caller_id" (1-M systems scores data same day), or an ID for a target database for WriteBackFunction, or an external way to see "Customer_ID"
to_score_folder_batch, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder, my_unique_folder_customerID)

print("-> INJECT static guid-folder")
print("")
print("E)INFERENCE BATCH: GOLD to score (example batch, datetime from config)")
print(to_score_folder_batch + 'gold_to_score.parquet') # + self.date_scoring_folder.strftime('%Y_%m_%d') + '/')
print("")
print("E)INFERENCE BATCH: GOLD scored (example batch, datetime from config)")
print(scored_folder +  'scored_*.parquet') 
print("")

-> INJECT static guid-folder

E)INFERENCE BATCH: GOLD to score (example batch, datetime from config)
projects/project002/01_titanic_model_clas/inference/22/gold/dev/2021_01_21/aa2b9f27b75c47e180f4cb72630996c7/gold_to_score.parquet

E)INFERENCE BATCH: GOLD scored (example batch, datetime from config)
projects/project002/01_titanic_model_clas/inference/22/scored/dev/2021_01_21/aa2b9f27b75c47e180f4cb72630996c7/scored_*.parquet



# Disaster recovery scenario
- Disaster: If pipeline has not been running for days, we need to score data back in time, and do WriteBack to that data maybe
- ESML allows to have 2 datefolders: "Today WEDNESDAY I cored data from last MONDAY"

In [21]:
print("C) INFERENCE BATCH (SCENARIO 2): TODAY I scored data from X days AGO  (second datefolder from config - X days ago)")
print(" \n")
print(to_score_folder_batch + p.date_scoring_folder.strftime('%Y_%m_%d') + '/')
print(scored_folder + p.date_scoring_folder.strftime('%Y_%m_%d') + '/')

C) INFERENCE BATCH (SCENARIO 2): TODAY I scored data from X days AGO  (second datefolder from config - X days ago)
 

projects/project002/03_diabetes_model_reg/inference/1/gold/dev/2021_06_08/65c9531aa0bd4a8b8c15dfd2163b1629/2021_06_08/
projects/project002/03_diabetes_model_reg/inference/1/scored/dev/2021_06_08/65c9531aa0bd4a8b8c15dfd2163b1629/2021_06_08/


# 1) SCORING PIPELINE (remote CPU cluster)
### 3 Parameters (environment, model_version, scoring_folder_date)
- Parameters from "calling orchestrator" can be either 
    - A) written to Datalake json, before the pipeline is triggered.
    - B) passed to the AML pipline (Azure Datafactory passes 3 parameters to this AML scoring pipeline)
- NB! The default behaviour is for ESML to read these 3 parameters from datalake "active" folder (json file) "project002/modelA/inference/active/*.json" at ESMLProject.init()
    - But, these 3 parameters can be passed directly, which will override the ones in the DataLake
        - A) passing them to the ESMLProject constructor (Use case: Debugging purpose)
        - B) passing them on the Python command line as `STRINGS`, to let ESML read from ArgumentParser (Use case: From Azure Devops Build & Release piplines, or from Azure Datafactory AML activity)
            - ESMLProject.get_project_from_env_command_line() will then do the below. 
                - `parser.add_argument('--esml_environment', type=str, help='ESML target environment: dev,test,prod')`
                - `parser.add_argument('--esml_inference_model_version', type=str, help='Model version to score with')`
                - `parser.add_argument('--esml_scoring_in_datetime', type=str, help='IN folder, datetype:datetime - the data to SCORE date_folder')`
                - `parser.add_argument('--esml_train_in_datetime', type=str, help='IN folder. datetype:datetime - the data to RETRAIN model on')`
            - NB! If `esml_inference_model_version` is `not set/None/Null or negative`...ESML will be ready for "retraining" rather than "inference/scoring"
            - NB! If `esml_inference_model_version`="`0`"...ESML will be use the highest&latest model version, to score with.

# Override `ACTIVE data & model` from Lake-config, with ArgParse and ESMLProject constructor

In [37]:
# Parameters from "calling orchestrator" = Azure Data factory as ab AML pipline activity 
# Note: The default behaviour is for ESML to read these parameters from datalake "active" folder (json file) "project002/modelA/inference/active/*.json" 
# But these can be passed directly, to override that, as seen below:

param_esml_env = "dev"
param_inference_model_version = "4"
param_scoring_folder_date = "2021-06-16 15:35:01.243860" # will become both IN and GOLD path:  
# inference/1/gold/dev/2021_06_08/1a51b4c19bec4409b61c52fa516e5aa2/
# inference/1/scored/dev/2021_06_08/1a51b4c19bec4409b61c52fa516e5aa2/
param_train_in_folder_date = "2021-01-16 15:35:01.243860" # 

optional_param_my_unique_scoring_folder = "22698b40d7d74d8c92b5fd8891d8a009"

In [38]:
#lakestore = p.set_lake_as_datastore(ws)
#p.readActiveDatesFromLake()

In [53]:
p = ESMLProject(param_esml_env,param_inference_model_version,param_scoring_folder_date,param_train_in_folder_date)
p.ws = p.get_workspace_from_config()
p.describe()

json_date_in_folder {'dev_in_folder_date': '2021-01-16 15:35:01.243860', 'test_in_folder_date': '2021-01-21 15:35:01.243860', 'prod_in_folder_date': '2020-01-01 15:35:01.243860'}
json_date_scoring_folder {'dev_scoring_folder_date': '2021-06-16 15:35:01.243860', 'test_scoring_folder_date': '2021-01-21 15:35:01.243860', 'prod_scoring_folder_date': '2020-01-01 15:35:01.243860', 'dev_inference_model_version': '4', 'test_inference_model_version': '22', 'prod_inference_model_version': '0'}
Inference version: 4

 - ds01_titanic
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/in/dev/2021/06/16/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/bronze/dev/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/out/silver/dev/
projects/project002/01_titanic_model_clas/inference/4/ds01_titanic/in/dev/2021/06/16/
 

Training GOLD (p.GoldPath)
projects/project002/01_titanic_model_clas/inference/4/gold/dev/
 

[A) USAGE]: to_score_folder, scored_fo

In [40]:
print("B)INFERENCE BATCH - LATEST scoring that day")
print(p.GoldPathToScoreBatch + 'gold_to_score.parquet')

B)INFERENCE BATCH - LATEST scoring that day
projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/gold_to_score.parquet


In [42]:
import uuid
my_unique_folder_customerID = uuid.uuid4().hex

In [60]:
to_score_folder_batch, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder, my_unique_folder_customerID)
print("to_score: ", to_score_folder_batch + 'gold_to_score.parquet' )
print("save_scored: ", scored_folder + 'scored_caller_id.parquet' )

to_score:  projects/project002/01_titanic_model_clas/inference/4/gold/dev/2021_06_16/fee9f4ebf405423c80a9d1377399f821/gold_to_score.parquet
save_scored:  projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_16/fee9f4ebf405423c80a9d1377399f821/scored_caller_id.parquet


In [57]:
from azureml.core.dataset import Dataset 
ds_to_score1 = Dataset.get_by_name(workspace = p.ws, name =  "M01_GOLD_TO_SCORE") # Opton 1) Azure ML Dataset directly
ds_scored = Dataset.get_by_name(workspace = p.ws, name =  "M01_GOLD_SCORED") # Opton 2) This would be static for this model, always same, fetches latest version
ds_scored


{
  "source": [
    "('project002', 'projects/project002/01_titanic_model_clas/inference/4/scored/dev/2021_06_16/af142621005247d4b9384f3da652e21e/*.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "6b5c76bf-4e31-43d5-bf43-51313e66d76d",
    "name": "M01_GOLD_SCORED",
    "version": 1,
    "description": "Scored gold data with model version 4",
    "tags": {
      "date_time_folder": "2021_06_16",
      "caller_id": "caller_id",
      "model_version": "4",
      "model": "01_titanic_model_clas"
    },
    "workspace": "Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG')"
  }
}

# 2) WriteBack PIPELINE (remote CPU cluster)

# END

# EXTRA - more about `AutoLake Paths`


In [7]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject
p = ESMLProject() 
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace

In [10]:
p.inference_mode = True # This flag will "change the paths", from TRAIN folder to INFERENCE folder-structure

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath)
    print("Bronze", d.BronzePath)
    print("Silver", d.SilverPath)

to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path(p.date_scoring_folder)
print("Gold", to_score_folder, "  ...uuid folder, is to be able to have multiple unique scorings, same datetime")

to_score_folder_latest, scored_folder_latest, date_folder_latest = p.get_gold_scored_unique_path(batch_datetime_from_config = None, unique_uuid4 = None, same_guid_folder=False)
print()
print("Gold (latest that day)", to_score_folder_latest, " ...No uuid folder, most recent scoring that day")

print("")
print("TRAIN")
print("")

p.inference_mode = False # This flag will "change the paths"

for d in p.Datasets:
    print(d.Name)
    print("IN", d.InPath)
    print("Bronze", d.BronzePath)
    print("Silver", d.SilverPath)

print("Gold", p.GoldPath)

ds01_titanic
IN projects/project002/01_titanic_model_clas/inference/1/ds01_titanic/in/dev/2021/01/01/
Bronze projects/project002/01_titanic_model_clas/inference/1/ds01_titanic/out/bronze/dev/
Silver projects/project002/01_titanic_model_clas/inference/1/ds01_titanic/out/silver/dev/
Gold projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_01_01/0a2b95ef81fe4f57b0ee4f7eb23c2080/   ...uuid folder, is to be able to have multiple unique scorings, same datetime

Gold (latest that day) projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_06_16/  ...No uuid folder, most recent scoring that day

TRAIN

ds01_titanic
IN projects/project002/01_titanic_model_clas/train/ds01_titanic/in/dev/2021/01/01/
Bronze projects/project002/01_titanic_model_clas/train/ds01_titanic/out/bronze/dev/
Silver projects/project002/01_titanic_model_clas/train/ds01_titanic/out/silver/dev/
Gold projects/project002/01_titanic_model_clas/train/gold/dev/


In [32]:
import datetime
print("FLEXIBILITY: If BATCH scoring (date_folder is read from config) && only 'LATEST SCORING that day' is of interest:")
print("- BUILD your SCORING & WriteBack pipeline, as per your liking (if you don't want support for multiple scorings per day)")
print("- Note: You need to register your own Azure ML Datasets, if not using the ESML pipeline Scoring&Writeback piplines.")
print()

print(" - Date folder only (no guid):")
dt_str = "2021-02-02 15:35:01.243860"
batch_datetime_from_config = datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S.%f') # DateTime
to_score_folder_latest, scored_folder_latest, date_folder_latest = p.get_gold_scored_unique_path(batch_datetime_from_config, None,False)
print("Gold to score (latest that day)", to_score_folder_latest)

print()

print(" - Guidfolder, but static")
guid_folder = 'e2fa50410ca343d8a620e55bce4fb244'
to_score_folder_latest, scored_folder_latest, date_folder_latest = p.get_gold_scored_unique_path(batch_datetime_from_config, guid_folder)
print("Gold to score (static GUID folder that day)", to_score_folder_latest)

FLEXIBILITY: If BATCH scoring (date_folder is read from config) && only 'LATEST SCORING that day' is of interest:
- BUILD your SCORING & WriteBack pipeline, as per your liking (if you don't want support for multiple scorings per day)
- Note: You need to register your own Azure ML Datasets, if not using the ESML pipeline Scoring&Writeback piplines.

 - Date folder only (no guid):
Gold to score (latest that day) projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_02_02/

 - Guidfolder, but static
Gold to score (static GUID folder that day) projects/project002/01_titanic_model_clas/inference/1/gold/dev/2021_02_02/e2fa50410ca343d8a620e55bce4fb244/


# More info - How to write ANY file to ADLS gen2

In [ ]:
#p.LakeAccess.upload(local_file_name, srs_folder, target_path, overwrite=True,use_dataset_factory = False) # BLOB or GEN 2